In [2]:

import numpy as np
import pandas as pd
import tensorflow as tf

from prepared_data.get_prepared_data import get_prepared_data
from prepared_data.train_model import split_features_target_and_map_target, save_results, callbacks


2022-05-16 08:50:55.138561: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 08:50:55.138598: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Load and prepard data

In [3]:
train_org = pd.read_csv('data/train.csv')
train, _, _ = get_prepared_data(train_org, map_target=False, number_of_history_matches=4)
train = pd.read_csv('data/train_made.csv', index_col=0)
train

/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


percent of object with nan value and orginals:  0.00, 92.767131


['target',
 'home_team_history_match_date_1',
 'home_team_history_is_play_home_1',
 'home_team_history_is_cup_1',
 'home_team_history_goal_1',
 'home_team_history_opponent_goal_1',
 'home_team_history_rating_1',
 'home_team_history_opponent_rating_1',
 'home_team_history_coach_1',
 'away_team_history_match_date_1',
 'away_team_history_is_play_home_1',
 'away_team_history_is_cup_1',
 'away_team_history_goal_1',
 'away_team_history_opponent_goal_1',
 'away_team_history_rating_1',
 'away_team_history_opponent_rating_1',
 'away_team_history_coach_1',
 'home_team_history_gol_difference_1',
 'home_team_mean_regeneration_time',
 'away_team_history_gol_difference_1',
 'away_team_mean_regeneration_time',
 'home_team_mean_hist_rat',
 'home_team_mean_all_ratting',
 'away_team_mean_hist_rat',
 'away_team_mean_all_ratting',
 'league_id_ratting',
 'sum_history_targets',
 'home_team_history_target_1_-1.0',
 'home_team_history_target_1_0.0',
 'away_team_history_target_1_-1.0',
 'away_team_history_targ

Classify function

In [ ]:

def build_and_compile_model(shape, n_neurons, dropout, learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(
            n_neurons, activation='relu',
            input_shape=shape),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(3)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model


Find model parameters and save model

In [5]:
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import operator
from sklearn.model_selection import StratifiedKFold

train, _, _ = get_prepared_data(train_org, map_target=False, number_of_history_matches=4)

for dropout in range(5, 6):

    dropout = 0.3
    n_neurons = 16
    learning_rate = 0.001
    batch_size = None

    scaler = MinMaxScaler()
    train_features, target = split_features_target_and_map_target(train)

    dnn_model = build_and_compile_model((train_features.shape[-1],), n_neurons, dropout, learning_rate)

    model_name = f'final/{4}_MinMax_{n_neurons}_d_{dropout}_{n_neurons}' + datetime.now().strftime("%Y:%m:%d-%H:%M:%S")
    output_path = 'evaluate_results/' + model_name

    save_results(output_path, f'lr={learning_rate} batch={batch_size}', dnn_model.summary)

    i = 0
    max_value_val_acc = list(range(0, 5))
    min_value_val_loss = list(range(0, 5))
    train_acc = list(range(0, 5))
    train_loss = list(range(0, 5))

    for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True, random_state=33).split(train_features,
                                                                                                    target):
        train_features_fold = scaler.fit_transform(train_features.iloc[train_index])
        val_features_fold = scaler.transform(train_features.iloc[test_index])

        model_full_name = model_name + f'/folds_{i}'

        early_stop, reduce_lr, tensorboard_callback, checkpoint_callback = callbacks(model_full_name)

        history = dnn_model.fit(
            train_features_fold,
            target.iloc[train_index],
            validation_data=(val_features_fold, target.iloc[test_index]),
            verbose=0, epochs=50,
            callbacks=[early_stop, reduce_lr, checkpoint_callback, tensorboard_callback],
            batch_size=batch_size
        )

        max_index_val_acc, max_value_val_acc[i] = max(enumerate(history.history['val_accuracy'], ),
                                                      key=operator.itemgetter(1))
        train_acc[i] = history.history['accuracy'][max_index_val_acc]

        min_index_val_loss, min_value_val_loss[i] = min(enumerate(history.history['val_loss'], ),
                                                        key=operator.itemgetter(1))
        train_loss[i] = history.history['loss'][min_index_val_loss]

        out_string = f'f_{i}\nval_acc={max_value_val_acc[i]} train_acc={train_acc[i]} ep={max_index_val_acc + 1}\nval_loss={min_value_val_loss[i]} train_loss={train_loss[i]} ep={min_index_val_loss + 1}'

        save_results(output_path, out_string)
        i += 1

        tf.keras.backend.clear_session()
        dnn_model = build_and_compile_model((train_features.shape[-1],), n_neurons, dropout, learning_rate)

        # dnn_model = build_and_compile_model(normalizer,n_neurons,dropout,learning_rate)

    out_string = f'median\nval_acc={np.median(max_value_val_acc)} train_acc={np.median(train_acc)}\nval_loss={np.median(min_value_val_loss)} train_loss={np.median(train_loss)}'

    save_results(output_path, out_string)


percent of object with nan value and orginals:  0.00, 92.767131


2022-05-06 23:47:36.366292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-06 23:47:36.366320: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-06 23:47:36.366337: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Zenon): /proc/driver/nvidia/version does not exist
2022-05-06 23:47:36.366528: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 00010: early stopping
Epoch 00011: early stopping
Epoch 00015: early stopping
Epoch 00011: early stopping
Epoch 00006: early stopping
